In [6]:
### A few options:
## https://github.com/sowole-aims/chat-with-documents-app

## 4.21 So far all I've figured out how to do is embed markdown with Chroma.  Need to actually figure out how to properly do querying

In [1]:
import os
import chromadb
from chromadb.utils import embedding_functions

def load_journal_entries(directory_path):
    journal_data = {}
    files = os.listdir(directory_path)

    for filename in files:
        if filename.endswith('.md'):
            # Use the filename without the extension as the key
            title = filename[:-3]  # Remove the .md extension
            
            # Read the file content
            filepath = os.path.join(directory_path, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
            
            journal_data[title] = content

    return journal_data

# Example of loading data from a specified directory
journal_data = load_journal_entries('/Users/danstern/Desktop/Roam/Export_Test')


In [2]:
client = chromadb.Client()

### Added this because it was causing runtime issues.  See here: https://github.com/chroma-core/chroma/issues/2013
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
collection = client.create_collection(name="all-my-documents", embedding_function=sentence_transformer_ef)

/Users/danstern/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Prep my data

value_of_content = list(journal_data.values())
titles = list(journal_data.keys())
metadata_titles = [{"title": value} for value in titles]


In [4]:
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=value_of_content,
    metadatas=metadata_titles, # filter on these!
    ids=titles # unique for each doc
)

In [5]:
collection.count()

3